In [4]:
%reset 


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
%run  ./Kickit_app_transformers.py

In [6]:

#from Kickit_app_transformers import Kickit_dataframe_preprocessor
#from Kickit_app_transformers import Kickit_frequency_encoder
#from Kickit_app_transformers import Kickit_weight_of_evidence_encoder
#from Kickit_app_transformers import  Kickit_dataframe_balancer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import warnings
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# Reading the Data set

In [7]:
# reading the dataset
df_raw = pd.read_csv('kick.csv')
df_raw.head(2)

,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,0,1260144000,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,...,11597,12409,?,?,21973,33619,FL,7100,0,1113
1,0,1260144000,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,...,11374,12791,?,?,19638,33619,FL,7600,0,1053


Extracting the axis of studies for the project custom to  ADESA website 

In [8]:
choosen_axis = ["Model", "Make" ,"Transmission","VehOdo","MMRAcquisitionAuctionAveragePrice", "IsBadBuy"]
df_adesa_raw = df_raw[choosen_axis]
df_adesa_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Model                              72983 non-null  object
 1   Make                               72983 non-null  object
 2   Transmission                       72983 non-null  object
 3   VehOdo                             72983 non-null  int64 
 4   MMRAcquisitionAuctionAveragePrice  72983 non-null  object
 5   IsBadBuy                           72983 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 3.3+ MB


# Pre-processing Pipeline for the freqency encoding technique

copying the raw data

In [9]:
df = df_adesa_raw.copy()

 Instantiating the preprocessing pipeline objects

In [10]:
# this on will clean the data with respect to our strategy
f_preprocess = Kickit_dataframe_preprocessor()
# this one will balance the dataset
f_balancer = Kickit_dataframe_balancer()
# this one will use the frequency encoding technique for categorical variable 
freq_encoder = Kickit_frequency_encoder()

Creating a pipeline for the first preprocessing technique :based on count of  frequency encoder

In [11]:
freq_pipeline = Pipeline([("preprocess",f_preprocess),("balancer", f_balancer),
                          ("fencoder",freq_encoder)])

preprocessing the dataframe with the count of frequency pipeline

In [12]:
adesa_freq_pipe = freq_pipeline.fit_transform(df)

checking the transformation for success

In [13]:
adesa_freq_pipe.head(5)

,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy
0,42,269,17360,89046,8155.0,0
1,236,2947,17360,93593,6854.0,0
2,139,2947,17360,73807,3202.0,0
3,136,2947,17360,65617,1893.0,0
4,246,3015,591,69367,3913.0,0


In [14]:
adesa_freq_pipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17952 entries, 0 to 17951
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Model                              17952 non-null  int64  
 1   Make                               17952 non-null  int64  
 2   Transmission                       17952 non-null  int64  
 3   VehOdo                             17952 non-null  int64  
 4   MMRAcquisitionAuctionAveragePrice  17952 non-null  float64
 5   IsBadBuy                           17952 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 841.6 KB


###  Modelization based on frequency count

 preparing and spliting the dataset for the modelization

In [15]:
features = adesa_freq_pipe.drop("IsBadBuy", axis =1)
target = adesa_freq_pipe["IsBadBuy"]
x_ftrain, x_ftest, y_ftrain, y_ftest = train_test_split(features, target, test_size=0.25, random_state=0)

Creating a list of pipelines for three different candidate classifiers 

In [16]:
f_scaler = StandardScaler()
f_randomForest = RandomForestClassifier()
f_logisticRegr = LogisticRegression(random_state=0)
f_decisionTree = DecisionTreeClassifier()

freq_bench_cross = [Pipeline([("scaler",f_scaler), ('lr_classifier', f_logisticRegr)]),
                    Pipeline([('dt_classifier', f_decisionTree)]),
                    Pipeline([('rf_classifier', f_randomForest)])] 

In [17]:
# Dictionary of pipelines and classifier types for ease of reference
freq_pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'RandomForest'}

# Fit the pipelines
for pipe in freq_bench_cross:
    pipe.fit(x_ftrain, y_ftrain)

In [18]:
for i,model in enumerate(freq_bench_cross):
    print("{} with count of frequency Test Accuracy: {}".format(freq_pipe_dict[i],model.score(x_ftest,y_ftest)))

Logistic Regression with count of frequency Test Accuracy: 0.5877896613190731
Decision Tree with count of frequency Test Accuracy: 0.553698752228164
RandomForest with count of frequency Test Accuracy: 0.588903743315508


# Pipelines for the Weight of Evidence  encoding technique

### Preprocessing the dataframe with the weight of evidence pipeline

In [19]:
df = df_adesa_raw.copy()

 Instantiating the pipeline object

In [20]:
# this on will clean the data with respect to our requirements
w_preprocess = Kickit_dataframe_preprocessor()
# this one will balance the dataset
w_balancer = Kickit_dataframe_balancer()
# # this one will use the weight of evidence encoding technique for categorical variable 
woe_encoder = Kickit_weight_of_evidence_encoder()

#Creating a pipeline for the second  preprocessing technique :based on the weight of evidence encoder

In [21]:
woe_pipeline = Pipeline([("preprocess",w_preprocess),("balancer", w_balancer),
                         ("woe_encoder", woe_encoder),("2nd balancer", w_balancer)])

preprocessing the dataframe with the weight of evidence pipeline

In [22]:
# the next command  after this will raise an unecessary warning message, this will help keep the notebook clean
warnings.filterwarnings('ignore')

In [23]:
adesa_woe_pipe = woe_pipeline.fit_transform(df)

In [24]:
adesa_woe_pipe['IsBadBuy'].sum()

8151

In [25]:
# checking results
adesa_woe_pipe.head(5)

,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy,Model_encoded,Make_encoded,Transmission_encoded
0,MAZDA3,MAZDA,AUTO,89046,8155.0,0,0.376478,-0.254530,0.028232
1,1500 RAM PICKUP 2WD,DODGE,AUTO,93593,6854.0,0,-0.169179,0.195080,0.028232
2,STRATUS V6,DODGE,AUTO,73807,3202.0,0,0.480463,0.195080,0.028232
3,NEON,DODGE,AUTO,65617,1893.0,0,-0.601439,0.195080,0.028232
4,FOCUS,FORD,MANUAL,69367,3913.0,0,-0.547706,-0.294471,-0.011628


In [26]:

woe_cat_dict.get('DODGE')

0.19507998940161894

In [27]:
woe_cat_dict.get('MAZDA')

-0.25452986513488046

In [28]:
woe_cat_dict.get('MAZDA3')

0.37647757123491205

In [29]:
# Checking the dictionary from our encoding class created for each category
woe_cat_dict

{'1500 RAM PICKUP 2WD': -0.16917873135476025,
 '1500 RAM PICKUP 4WD': -0.3677247801253174,
 '1500 SIERRA PICKUP 2': 0.41985384556026406,
 '1500 SILVERADO PICKU': 0.742374121325096,
 '2500 RAM PICKUP 2WD': -0.5108256237659907,
 '2500HD SILVERADO PIC': -0.25131442828090605,
 '300': 0.7810085363512795,
 '300 2.7L V6 MPI': 1.0986122886681098,
 '300M': -1.0986122886681098,
 '300M 3.5L V6 MPI': -0.8472978603872037,
 'ACCENT': -0.04348511193973878,
 'ACCENT 1.6L I4 MPI': -0.4228568508200336,
 'ACCORD 4C': 0.5877866649021191,
 'ACCORD 4C 2.4L I-4 M': 0.8472978603872037,
 'ACCORD V6': 0.8209805520698303,
 'ACCORD V6 3.0L V6 EF': 0.5877866649021191,
 'ALERO 4C': 0.14842000511827322,
 'ALERO 4C 2.2L I4 MPI': -0.7884573603642702,
 'ALERO V6': -0.9555114450274363,
 'ALERO V6 3.4L V6 MPI': -0.6359887667199967,
 'ALTIMA': 0.17904823144898546,
 'ALTIMA 2.5L I4 EFI': -0.9382696385929302,
 'ALTIMA 2.5L I4 MPI': 0.23180161405732438,
 'AMANTI': 0.5877866649021191,
 'AMANTI 3.5L V6 MPI': -0.251314428280906

In [30]:
woe_cat_dict 
#the nan value are here can simply be compared to having no values or zero, we will set it in the flask code part
# using the np.nan_to_num method

{'1500 RAM PICKUP 2WD': -0.16917873135476025,
 '1500 RAM PICKUP 4WD': -0.3677247801253174,
 '1500 SIERRA PICKUP 2': 0.41985384556026406,
 '1500 SILVERADO PICKU': 0.742374121325096,
 '2500 RAM PICKUP 2WD': -0.5108256237659907,
 '2500HD SILVERADO PIC': -0.25131442828090605,
 '300': 0.7810085363512795,
 '300 2.7L V6 MPI': 1.0986122886681098,
 '300M': -1.0986122886681098,
 '300M 3.5L V6 MPI': -0.8472978603872037,
 'ACCENT': -0.04348511193973878,
 'ACCENT 1.6L I4 MPI': -0.4228568508200336,
 'ACCORD 4C': 0.5877866649021191,
 'ACCORD 4C 2.4L I-4 M': 0.8472978603872037,
 'ACCORD V6': 0.8209805520698303,
 'ACCORD V6 3.0L V6 EF': 0.5877866649021191,
 'ALERO 4C': 0.14842000511827322,
 'ALERO 4C 2.2L I4 MPI': -0.7884573603642702,
 'ALERO V6': -0.9555114450274363,
 'ALERO V6 3.4L V6 MPI': -0.6359887667199967,
 'ALTIMA': 0.17904823144898546,
 'ALTIMA 2.5L I4 EFI': -0.9382696385929302,
 'ALTIMA 2.5L I4 MPI': 0.23180161405732438,
 'AMANTI': 0.5877866649021191,
 'AMANTI 3.5L V6 MPI': -0.251314428280906

In [31]:
adesa_woe_pipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16302 entries, 0 to 16301
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Model                              16302 non-null  object 
 1   Make                               16302 non-null  object 
 2   Transmission                       16302 non-null  object 
 3   VehOdo                             16302 non-null  int64  
 4   MMRAcquisitionAuctionAveragePrice  16302 non-null  float64
 5   IsBadBuy                           16302 non-null  int64  
 6   Model_encoded                      16302 non-null  float64
 7   Make_encoded                       16302 non-null  float64
 8   Transmission_encoded               16302 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 1.1+ MB


### Modelization based on WOE 

In [32]:
features = adesa_woe_pipe
target = adesa_woe_pipe["IsBadBuy"]
x_wtrain, x_wtest, y_wtrain, y_wtest = train_test_split(features, target, test_size=0.25, random_state=0)

In [33]:
w_scaler = StandardScaler()
w_randomForest = RandomForestClassifier()
w_logisticRegr = LogisticRegression(random_state=0)
w_decisionTree = DecisionTreeClassifier()

# Creating a list of pipelines for three different candidate classifiers 
woe_bench_cross = [Pipeline([("scaler",w_scaler), ('lr_classifier', w_logisticRegr)]),
                    Pipeline([('dt_classifier', w_decisionTree)]),
                    Pipeline([('rf_classifier', w_randomForest)])] 

In [34]:
x_wtrain

,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy,Model_encoded,Make_encoded,Transmission_encoded
5997,LIBERTY 2WD V6,JEEP,AUTO,75105,4894.0,1,0.467341,-0.194642,0.028232
16011,TAURUS 3.0L V6 EFI,FORD,AUTO,60451,2239.0,1,-0.462272,-0.294471,0.028232
6335,GRAND CHEROKEE 2WD V,JEEP,AUTO,83173,13403.0,0,-0.283575,-0.194642,0.028232
8466,TOWN & COUNTRY FWD V,CHRYSLER,AUTO,81901,5070.0,0,0.029632,-0.167054,0.028232
8402,IMPALA 3.5L V6 SFI,CHEVROLET,AUTO,72926,7605.0,0,0.998529,0.358029,0.028232
...,...,...,...,...,...,...,...,...,...
13123,L SERIES,SATURN,AUTO,52219,4162.0,1,-1.031921,-0.253901,0.028232
3264,IMPALA V6 3.5L V6 SF,CHEVROLET,AUTO,71720,9216.0,1,0.902868,0.358029,0.028232
9845,STRATUS 4C 2.4L I4 M,DODGE,AUTO,71372,2092.0,1,0.263417,0.195080,0.028232
10799,IMPALA V6,CHEVROLET,AUTO,79691,7883.0,1,1.484275,0.358029,0.028232


In [35]:
# Dictionary of pipelines and classifier types for ease of reference
woe_pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: ' Default mode RandomForest'}

# Fit the pipelines
for pipe in woe_bench_cross:
    pipe.fit(x_wtrain.drop(["IsBadBuy", "Model", "Make","Transmission"], axis =1), y_wtrain)

In [36]:
for i,model in enumerate(woe_bench_cross):
    print("{} with Weight of evidence Test Accuracy: {}".format(woe_pipe_dict[i],model.score(x_wtest.drop(["IsBadBuy", "Model", "Make","Transmission"], axis =1),y_wtest)))

Logistic Regression with Weight of evidence Test Accuracy: 0.6415603532875368
Decision Tree with Weight of evidence Test Accuracy: 0.5662414131501472
 Default mode RandomForest with Weight of evidence Test Accuracy: 0.6104023552502453


# Random frorest classifier Auto-Hyperparameters tuning

In [37]:
# Create a pipeline
pipe = make_pipeline((RandomForestClassifier()))
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                {"randomforestclassifier": [RandomForestClassifier()],
                 "randomforestclassifier__n_estimators": [10],
                 "randomforestclassifier__max_depth":[10,30,25,20],
                 "randomforestclassifier__min_samples_leaf":[1,2,5,6,10,15,100],
                 "randomforestclassifier__max_leaf_nodes": [2, 5,10]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=10, verbose=0,n_jobs=-1) # Fit grid search
Upgraded_Random_frorest = gridsearch.fit(x_wtrain.drop(["IsBadBuy", "Model", "Make","Transmission"], axis =1),y_wtrain)

In [38]:
Upgraded_Random_frorest.score(x_wtest.drop(["IsBadBuy", "Model", "Make","Transmission"], axis =1), y_wtest)

0.6449950932286556

In [39]:
print(Upgraded_Random_frorest.best_estimator_)

Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(max_depth=25, max_leaf_nodes=10,
                                        min_samples_leaf=10,
                                        n_estimators=10))])


# Results

The best pipeline between the frequency encoding and the weight of evidence encoding is  : Weight of evidence one 

The best model within that pipeline is : Logistic regression  then Upgraded Random forest models with the same score

saving the created encoding dictionnary for interpretation in the flask part

In [40]:
file = open('kidict.pkl', 'wb')
pickle.dump(woe_cat_dict, file)

saving the selected model

In [41]:
file = open('model.pkl', 'wb')
pickle.dump(Upgraded_Random_frorest, file)

# Test

In [42]:
a = Upgraded_Random_frorest.predict(x_wtest.drop(["IsBadBuy",
                                                  "Model",
                                                  "Make",
                                                  "Transmission"], axis =1))
a

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [43]:
len(a)

4076

In [44]:
a.sum()

2063

In [45]:
x_wtest['predicted_label'] = Upgraded_Random_frorest.predict(x_wtest.drop(["IsBadBuy",
                                                                           "Model", 
                                                                           "Make",
                                                                           "Transmission"], axis =1))

In [46]:
x_wtest.head(5) 


,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy,Model_encoded,Make_encoded,Transmission_encoded,predicted_label
3980,IMPALA V6,CHEVROLET,AUTO,67669,7881.0,0,1.484275,0.358029,0.028232,0
5473,GRAND PRIX,PONTIAC,AUTO,69286,6173.0,0,0.323903,0.105682,0.028232,0
13891,CALIBER 2.0L I4 SFI,DODGE,AUTO,47090,7245.0,1,0.336472,0.195080,0.028232,0
9653,IMPALA,CHEVROLET,AUTO,83743,4022.0,1,0.471178,0.358029,0.028232,1
2162,IMPALA,CHEVROLET,AUTO,64217,7293.0,0,0.471178,0.358029,0.028232,0


In [47]:
woe_cat_dict.get('IMPALA')

0.47117848741885987

In [48]:
test = x_wtest.drop(["Model_encoded", "Make_encoded","Transmission_encoded"], axis =1)
test.head(10)

,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy,predicted_label
3980,IMPALA V6,CHEVROLET,AUTO,67669,7881.0,0,0
5473,GRAND PRIX,PONTIAC,AUTO,69286,6173.0,0,0
13891,CALIBER 2.0L I4 SFI,DODGE,AUTO,47090,7245.0,1,0
9653,IMPALA,CHEVROLET,AUTO,83743,4022.0,1,1
2162,IMPALA,CHEVROLET,AUTO,64217,7293.0,0,0
8918,HIGHLANDER 2WD V6 3.,TOYOTA,AUTO,80726,7815.0,1,0
12321,FREESTAR FWD V6 3.9L,FORD,AUTO,46401,5832.0,1,1
1919,DURANGO 2WD V8,DODGE,AUTO,82522,0.0,0,1
3018,SEBRING 4C,CHRYSLER,AUTO,73142,7340.0,0,0
1358,GRAND PRIX,PONTIAC,AUTO,59598,6369.0,0,0
